# Publimed estilo Reuters

En el siguiente notebook vamos a utilizar el dataset publimed con el cual aplicaremos NMF, por lo que:
* El dataset corresponde a una recopilación de preguntas médicas sobre documentos de diversos tópicos médicos
* Vamos a pasar el dataset al estilo Reuters
    * Recordar que esto se refiere a una representacion a través de las palabras y frecuencias
* Utilzaremos sobre ella la descomposición NMF Rank-2
    * Nos enfocaremos en una clasificación sobre los tópicos de los documentos

### Librerías

In [1]:
import pandas as pd
from functools import reduce
import numpy as np
from random import randrange

## Cargo y manejo de los datos

* Convertiremos el dataset publimed que mantenemos almacenados en un documento CSV
* Leeremos los documentos y rescataremos los datos que nos interesan
* Codificaremos tanto su contenido como su etiqueta para dejarlo en formato "Reuters"
    * Esta se refiere a una representacion a través de las palabras del doc
    * La codificacion depende de la frecuencia de las palabras del vocabulario utilizado
* Este preceso busca poder convertir cualquier dataset a modo reuters

### Generar vocabulario

* Contaremos las palabras para generar el vocabulario que nos permitirán establecer la frecuencia

In [2]:
def cargar_datos(url):
  """
  Cargamos los datos del dataset publimed
  Retornamos un dataframe con los datos
  """
  return pd.read_csv(url, sep='|')

In [3]:
def puntuacion(text):
  """
  Recibe un texto y elimina los signos de puntuacion
  """
  forbidden = ("?", "¿", "¡", "!", ",", ".", ";", ":", "-", "[", "]", "'", "(", ")", "{", "}", '"', "/")
  texto = text.lower()
  aux = "" 
  for v in texto:
    if not v in forbidden:
      aux += v
  aux = aux.replace("  ", " ")
  return aux

def solo_oraciones(data, atributo="title"):
  """
  Obtiene una lista de las oraciones
  de los atributos seleccionados de data
  """
  return list(puntuacion(x) for x in set(data[atributo]) if type(x) == str)

def todas_oraciones(data, atributos=['title', 'abstract']):
  """
  Retorna todas las oraciones de la data
  que pertenecen a los atributos seleccioandos
  """
  return reduce(lambda x,y: x + solo_oraciones(data, y), atributos, list())

In [4]:
def word_count(counts, string):
  """
  Recibe un registro de los conteos de las palabras
  y un string.
  Actualiza el contador de las palabras y lo retorna
  """
  words = string.split()
  for word in words:
    if word in counts:
      counts[word] += 1
    else:
      counts[word] = 1
  return counts


def obtener_palabras(data, atributos=['title', 'abstract']):
  """
  Recibe el data completo
  Retorna un arreglo de todas las palabras
  de los atributos que se deseen
  """
  oraciones, palabras = todas_oraciones(data, atributos), dict()
  oraciones = list(set(oraciones))
  for frase in oraciones:
    palabras = word_count(palabras, frase)
  return palabras


def construir_vocabulario(palabras):
  """
  Cuenta la cantidad de veces que son utilizadas las palabras
  retorna un arreglo de las palabras con su frecuencia
  """
  keys = ['word', 'frequency']
  vocabulario = list(dict(zip(keys, tupla)) for tupla in palabras.items())
  return sorted(vocabulario, key = lambda i: i['frequency'], reverse=True)


def asignar_id(vocabulario):
  """
  Agrega un id para las palabras utilizadas
  """
  for idx, palabra in enumerate(vocabulario):
    palabra['id'] = idx + 3
  return vocabulario


def generar_vocabulario(data):
  """
  A partir de la data determina las palabras y genera el vocabulario
  para realizar la configuracion de los datos
  """
  palabras = obtener_palabras(data)
  vocabulario = construir_vocabulario(palabras)
  return asignar_id(vocabulario)

### Realizar codificiacion

* A partir del vocabulario generado que incluye la frecuencia de uso de las palabras
* Generamos la configuracion de los datos utilizados
* Esto nos permitirá obtener una representacion de cada documento

In [5]:
def generar_labels(data):
  registro, topicos, keys = dict(), list(), ['topic', 'frequency']

  for index, row in data.iterrows():
    topico = row['topic_name']
    if topico in registro:
      registro[topico] += 1
    else:
      registro[topico] = 1

  for tupla in registro.items():
    tupla_actual = dict(zip(keys, tupla))
    topicos.append(tupla_actual)

  topicos = sorted(topicos, key = lambda i: i['frequency'], reverse=True)

  retorno = dict()
  for idx, topico in enumerate(topicos):
    topico['id_label'] = idx+1
    retorno[topico['topic']] = idx+1
    #print(topico['topic'])
  return topicos

In [6]:
def encontrar_topic(nombre, label_topic):
  for idx, elemento in enumerate(label_topic):
    if nombre == elemento['topic']:
      return idx+1

def id_topicos(data, label_topic):
    return list( encontrar_topic(row['topic_name'], label_topic) for _, row in data.iterrows() )

In [7]:
def unir_contenido(row):
  if row['code_abstract'] is None:
    return row['code_title']
  return row['code_title'] + row['code_abstract']

def generar_contenido(data):
  return list(unir_contenido(row) for _, row in data.iterrows())

def generar_dataset(data):
  retorno = dict()
  label_topic = generar_labels(data)
  retorno['content'] = generar_contenido(data)
  retorno['label'] = id_topicos(data, label_topic)
  return retorno

### Manejo de variables para dataset

* Generamos opciones para poder manejar el dataset y su separacion

In [8]:
def filtro_ranking(elemento, num_words, skip_top):
  retorno = list()
  maximo = num_words + skip_top
  for x in elemento:
    if skip_top < x < maximo:
      retorno.append(x)
  return retorno

def filtros(content, max_len, num_words, skip_top):
  retorno = list()
  for elemento in content:
    if num_words and elemento:
      elemento = filtro_ranking(elemento, num_words, skip_top)
    if max_len and elemento:
      elemento = elemento[:max_len]
    retorno.append(elemento)
  return retorno


In [9]:
def encontrar_id(vocabulario, palabra, atributo, idx):
  """
  Encontramos el id que posee la palabra en el vocabulario
  """
  for tupla in vocabulario:
    if tupla['word'] == palabra:
      return(tupla['id'])
  return None

def to_code(vocabulario, oracion, atributo, idx):
  """
  Codificamos las frases utilizadas para el atributo
  """
  oracion = oracion.strip().split()
  return list(encontrar_id(vocabulario, palabra, atributo, idx) for palabra in oracion)

def code_atributo(data, vocabulario, atributo):
  """
  Codifica el atributo a partir de la representacion
  a partir de la frecuencia suada en ese atributo
  """
  columna_atributo = list()
  for index, row in data.iterrows():
    if type(row[atributo]) is str:
      auxiliar = to_code(vocabulario, puntuacion(row[atributo]), atributo, index)
    else:
      row[atributo], auxiliar = None, None
    columna_atributo.append(auxiliar)
  data[f'code_{atributo}'] = columna_atributo
  return data

def code_atributos(data, vocabulario, atributos=['title', 'abstract']):
  """
  Codifica los atributos de la data para
  una representacion de frecuencia de palabras
  """
  for atributo in atributos:
    data = code_atributo(data, vocabulario, atributo)
  return data

In [10]:
def load_data(data, vocabulario, test_split, max_len=None, num_words=None, skip_top=None):
  #data = cargar_datos(url)
  #vocabulario = generar_vocabulario(data)
  data_procesada = code_atributos(data, vocabulario)
  dataset = generar_dataset(data_procesada)
  largo = len(dataset['content'])
  len_train = int(largo*test_split)
  #len_test = largo - len_train
  
  #print(largo, len_train, test_split)
  
  dataset['content'] = filtros(dataset['content'], max_len, num_words, skip_top)
  
  train = np.array(dataset['content'][len_train:]), np.array(dataset['label'][len_train:])
  test = np.array(dataset['content'][:len_train]), np.array(dataset['label'][:len_train])
  #print(len(train), len(test))
  return train, test

### Edición del dataset

* Generamos funciones para poder representar mejor el dataset

In [11]:
def eliminar_repetidos(x_train, y_train):
  x_retorno, y_retorno = list(), list()
  
  for i, elemento in enumerate(x_train):
    if x_retorno and elemento == x_retorno[-1]:
      y_retorno[-1].append(y_train[i])
      continue
    x_retorno.append(elemento)
    y_retorno.append([y_train[i], ])
  return x_retorno, y_retorno

def mensaje_code(vocabulario, code):
  retorno = list( to_word(x, vocabulario) for x in code)
  return " ".join(retorno)

def to_word(idx, vocabulario):
  return vocabulario[idx-3]['word']

In [12]:
def eliminar_vacios(x_train, y_train):
  """
  Por diferentes opciones pueden quedar valores vacios
  Eliminamos esos valores vacios
  """
  x_retorno, y_retorno = list(), list()
  for i, x in enumerate(x_train):
    if x:
      x_retorno.append(x)
      y_retorno.append(y_train[i])
  return x_retorno, y_retorno

### Carguemos los datos

* Vamos a cargar los datos según caracteristicas que hagan más fácil de entender el resultado

In [13]:
# PARAMETROS

num_words = 100 # solo tomaremos las 100 primeras palabras
skip_top = 15 # palabras que no consideraremos
max_len = None # N° de palabras vocabulario
test_split = 0.05 # Porcentaje de palabras en el test set
url = '../dataset/publimed_medium.csv'

In [14]:
def mapping_labels(labels):
  return list(x['topic'] for x in labels)

In [15]:
data = cargar_datos(url)
vocabulario = generar_vocabulario(data)
label_topic = mapping_labels(generar_labels(data))

(x_train, y_train), (x_test, y_test) = load_data(data,
                                                 vocabulario,
                                                 test_split=test_split,
                                                 max_len=max_len,
                                                 num_words=num_words,
                                                 skip_top=skip_top)

9

In [16]:
print(f'Tamaño train set: {x_train.shape}, {y_train.shape}')
print(f'Tamaño test set: {x_test.shape}, {y_test.shape}')
print(f'Cantidad clases: {max(max(y_train), max(y_test))}')

Tamaño train set: (3325,), (3325,)
Tamaño test set: (175,), (175,)
Cantidad clases: 9


In [17]:
x_train, y_train = eliminar_vacios(x_train, y_train)

In [18]:
n_clases = max(y_train)
n_clases

9

In [19]:
for label in generar_labels(data):
  print(f"id: {label['id_label']} - freq: {label['frequency']}")

id: 1 - freq: 1152
id: 2 - freq: 1146
id: 3 - freq: 1144
id: 4 - freq: 53
id: 5 - freq: 1
id: 6 - freq: 1
id: 7 - freq: 1
id: 8 - freq: 1
id: 9 - freq: 1


## Aplicamos Rank-2

Ya que tenemos el dataset listo para poder trabajar en estilo "Reuters"

* Vamos a importar las funciones del tutorial del uso de reuters
* Utilizamos las funciones para poder separar en ciertos grupos

### Importamos las funciones

In [20]:
import ipynb.fs.full.reuters_functions as reu
import numpy as np

Using TensorFlow backend.


In [21]:

def conteo_labels(n, p):
  """
  Para una mejor representacion de los labels
  utilizaremos medidores para que no supere ciertas barreras
  """
  return list( [0, randrange(p-5, p+5)] for _ in range(n) )


def reducir_labels(data_array, labels, k=7, pivote=25):
  """
  Filtramos los documentos que pertenezcan a las primeras k clases
  Retornamos el arreglo con los documentos y sus correspondientes labels
  """
  
  conteo = conteo_labels(max(labels), pivote)
  retorno, retorno_labels = list(), list()
  for i in range(len(data_array)):
    if labels[i] < k and conteo[labels[i]][0] < conteo[labels[i]][1]:
      retorno.append(data_array[i])
      retorno_labels.append(labels[i])
      conteo[labels[i]][0] += 1
  return np.array(retorno), retorno_labels

In [22]:
x_data, y_data = reducir_labels(x_train, y_train, k=n_clases, pivote=25)

### Pasar los datos a matriz doc-term

In [23]:
# Importamos tokenizer para producirlo

from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical

tkn = Tokenizer(num_words=num_words) # tamaño del vocabulario
num_clases = max(y_data) + 1

In [24]:
# Contruimos la matriz pasando los datos a binario

x_data_bin = tkn.sequences_to_matrix(x_data, mode='binary')
y_data_cat = to_categorical(y_data, num_clases)

In [25]:
# Revisamos los como queda la representacion

entry = 10
print(f'Train Set \n\n  Values: {x_data_bin.shape}    n° docs x n° words \n  Ej: {x_data_bin[entry]}\n')
print(f'\n  Labels: {y_data_cat.shape}   (usaremos este) \n {y_data_cat[entry]}')

Train Set 

  Values: (106, 100)    n° docs x n° words 
  Ej: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0.
 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


  Labels: (106, 9)   (usaremos este) 
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [26]:
# Siguiendo la interpretación del paper

A_matrix = np.transpose(x_data_bin)

## Presentación de datos

In [27]:
def obtener_palabras_destacadas(lista_arbol, index_to_word, cantidad=10):
  for nodo in lista_arbol:
    if nodo['matrix'].shape[1] == 1:
      nodo['W'] = nodo['matrix']
    elif 'W' not in nodo.keys():
      nodo['W'], nodo['H'] = reu.calcular_descomposicion(nodo['matrix'])
    nodo['destacadas'] = reu.palabras_destacadas(nodo['W'], index_to_word, cantidad)
  return lista_arbol


def topicos_relevantes(y_data, columnas, mapping):
  """
  Ordena los topicos relevantes para cada nodo
  del arbol jerarquico
  """
  counts = dict()
  for col in columnas:
    if str(y_data[col]) in counts:
      counts[str(y_data[col])] += 1
    else:
      counts[str(y_data[col])] = 1
      
  keys = ['label', 'frequency']

  auxiliar = list(dict(zip(keys, tupla)) for tupla in counts.items())

  labels = sorted(auxiliar, key = lambda i: i['frequency'], reverse=True)
  
  for elemento in labels:
    elemento['label'] = int(elemento['label'])
    elemento['label_name'] = mapping[elemento['label']-1]
  return labels

def palabras_relevantes(palabras):
  """
  Selecciona las palabras más relevantes para cada nodo
  del arbol jerarquico
  """
  return list(x['word']['word'] for x in palabras[:5])


def presentar_nodos(lista_arbol, y_data, mapping):
  """
  Imprime una vision preeliminar de los nodos del arbol jerarquico
  """
  for nodo in lista_arbol:
    print(f"Nodo {nodo['id']}")
    print(f"  parent: {nodo['parent']} - leafs {nodo['hijos']}\n")
    topicos = topicos_relevantes(y_data, nodo['columnas'], mapping)
    for topico in topicos:
      print(f"label: {topico['label']}, frecuencia: {topico['frequency']}")
      print(f"  {topico['label_name']}")
    print("")
    palabras = palabras_relevantes(nodo['destacadas'])
    frase = ""
    for w in palabras:
      if frase:
        frase += " / "
      frase += w
    print(frase, "\n----------------------------------------\n")

## Aplicación NMF Rank-2

In [28]:
# Variables globales

numero_nodos = 4 # cantidad nodos para crear
beta = 1.1 # diferencia de tamaño mínima que habrá entre los nodos

In [29]:
tkn = Tokenizer(num_words=num_words) # tamaño del vocabulario
num_clases = max(y_data) + 1

In [30]:
lista_arbol = reu.jerarquizacion(A_matrix, vocabulario, 9)

In [31]:
#lista_arbol

In [32]:
#lista_arbol = obtener_palabras_destacadas(lista_arbol, vocabulario, 10)

In [33]:
#lista_arbol = reu.limpiar_lista(lista_arbol)

In [34]:
#presentar_nodos(lista_arbol, y_data, label_topic)

In [35]:
import operator

def top_words(palabras):
  arreglo_palabras = dict((x['word']['word'], x['word']['frequency']) for x in palabras)
  return dict(sorted(arreglo_palabras.items(), key=operator.itemgetter(1), reverse=True))

def arreglar_destacadas(lista_nodos):
  for nodo in lista_nodos:
    nodo['destacadas'] = top_words(nodo['destacadas'])
  return lista_nodos

In [36]:
lista_arbol = arreglar_destacadas(lista_arbol)

In [37]:
x = {'id': 2,
  'parent': 0,
  'matrix': None,
  'puntaje': 0.4374901583780595,
  'shape': (100, 16),
  'hijos': None,
  'columnas': [12, 13, 14, 15, 58, 59, 60, 72, 73, 74, 78, 80, 81, 84, 85, 88],
  'W': None,
  'H': None}

In [38]:
def seleccionar_caracteristicas(lista_nodos, caracteristicas):
  retorno = list()
  for elemento in lista_nodos:
    actual = dict()
    for x in caracteristicas:
      actual[x] = elemento[x]
    retorno.append(actual)
  return retorno

In [21]:
lista_arbol = seleccionar_caracteristicas(lista_arbol, ['id', 'parent', 'hijos', 'destacadas'])

NameError: name 'seleccionar_caracteristicas' is not defined

In [22]:
import json
json.dumps(lista_arbol)

NameError: name 'lista_arbol' is not defined